In [12]:
# Import modules
import torch
import numpy as np
torch.manual_seed(7)

In [13]:
# Create a parallel db generator which removes one record from the original db and keeps the rest of the records
def par_db_generator(db, idx):
    return torch.cat((db[0:idx], db[idx+1:]))

In [14]:
def coinflip(noise):
    answer = 0
    if torch.rand(1)>noise:
        answer=1
        return answer
    else:
        if torch.rand(1)>0.5:
            answer=1
            return answer
        return answer
            

In [15]:
# Backle up the parallel databases together
# it depends entirely on the length of the first db
def par_dbs(db):
    """This creates an array with the datase columns = len(db-1)
    Since we are removing one record at a time and returning the remaining values
    """
    db_all = []
    par_db = torch.ones(db.shape).type(torch.int)
    for i in range(len(db)):
        par_db = torch.cat((db[0:i], db[i+1:]))
        
        db_all.append(par_db)
        
    return db_all

In [16]:
# Create a function to generate the required data based on a given sample of users or records
def generate_db(rec_no):
    """This function generates the db that is then usd to generate the parallel dbs"""
    db = torch.rand(rec_no)>0.5
    # print('Original DB\n{}'.format(db))
    
    #Generate the entire parallel db
    # My Solution
    pdbs = par_dbs(db)
    
    
    # Lecture Solution
#     first_coin_flip = (torch.rand(len(db))>0.5).float()
#     second_coin_flip = (torch.rand(len(db))>0.5).float()
#     pdbs = db.float()*first_coin_flip + (1-first_coin_flip)*second_coin_flip
    
    return db, pdbs    

In [17]:
def query_sum(db):    
    return torch.sum(db)

In [18]:
def query_mean(db):
   
    return db.float().mean()

In [64]:
db, _ = generate_db(100)    

In [65]:
epsilon = 0.5

In [115]:
def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    print(query(db)) if query==query_sum else print(query_mean(db))
    return query(db) + noise

In [116]:
laplacian_mechanism(db, query_sum, 1)

tensor(47)


tensor([45.0633], dtype=torch.float64)

In [133]:
laplacian_mechanism(db, query_mean, 1/100)

tensor(0.4700)


tensor([0.4675], dtype=torch.float64)